In [1]:
import nltk
from nltk.corpus import brown
import numpy as np
from nltk.tag import map_tag
from collections import defaultdict, Counter

In [2]:
nltk.download('brown')
nltk.download('universal_tagset')

start_tag = '^'


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [3]:
data = brown.tagged_sents(tagset='universal')
# data = [(word.lower(),tag) for i in data for word, tag in i]
# train_size = int(len(data) * 0.8)
# train_data = data[:train_size]
# test_data = data[train_size:]

# train_data
K = 5
start = 0
end = int(len(data)/K) -1

emission_prob_list = []
transition_prob_list = []

data_fold_wise = []
for i in range(K):
  data_fold_wise.append(data[start:end])
  start += int(len(data)/K)
  end += int(len(data)/K)


In [4]:
tags = set(tag for sent in data for _, tag in sent)
sorted_tags = sorted(tags)
sorted_tags.append(start_tag)

In [5]:
prt_words = set(_ for sent in data for _, tag in sent if tag == "PRT")

In [6]:
prt_words

{"'ello",
 "'mon",
 "'tain't",
 "'tis",
 'Aaa-ee',
 'Aaawww',
 'Aah',
 'Ah',
 'Ah-ah',
 'Ahah',
 'Ahm',
 'Alas',
 'All',
 'Amen',
 "Arlene's",
 "Arthur's",
 'Atta',
 'Aw',
 'Aye-yah-ah-ah',
 'Bah',
 "Baseball's",
 'Be',
 "Bill's",
 "Black's",
 "Blackwell's",
 'Bong',
 'Boy',
 'Bravo',
 "Breed's",
 "Buckhorn's",
 'Bullshit',
 'Bye',
 "C'mon",
 "Celie's",
 'Chrissake',
 'Christ',
 'Creepers',
 "Crosson's",
 'Crunch',
 'Da-da-da-dum',
 'Dammit',
 'Down',
 'Eh',
 "Everything's",
 'Fiddlesticks',
 'Gawdamighty',
 'Gee',
 'Glory',
 'God',
 'Goddammit',
 'Goddamn',
 'Golly',
 'Good-by',
 'Good-bye',
 'Goodby',
 'Goodbye',
 'Goody',
 'Goolick',
 'Gosh',
 "Great's",
 "Guardino's",
 "Gyp'll",
 'Half',
 'Hallelujah',
 'Harro',
 'Haw',
 "He'd",
 "He'll",
 "He's",
 "Heat's",
 'Heigh-ho',
 'Hell',
 'Hello',
 'Henh',
 "Here's",
 'Hey',
 'Hi',
 'Hmm',
 'Hmpf',
 'Hoa-whup',
 'Hooray',
 "How'd",
 "How's",
 'Howda',
 'Howdy',
 'Hubba',
 'Hurrah',
 'Hurray',
 'Hush',
 "I'd",
 "I'll",
 "I'm",
 "I've",
 "Ik

In [7]:
sorted_tags

['.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X',
 '^']

In [8]:
def calc_emission_prob(data):
    word_tag_count = defaultdict(Counter)
    tag_count = Counter()
    emission_prob = defaultdict(dict)

    for sentence in data:
        for word, tag in sentence:
            word_tag_count[word.lower()][tag] += 1
            tag_count[tag] += 1

    for word in word_tag_count:
        for tag in word_tag_count[word]:
          emission_prob[word][tag] = (word_tag_count[word][tag] + 1) / (tag_count[tag] + len(word_tag_count))

          # if tag == "NUM" and (is_written_number(word) or is_number(word)):
          #   emission_prob[word][tag] = (word_tag_count[word][tag] + tag_count[tag]*0.8) / (tag_count[tag] + len(word_tag_count))
          # elif tag == "." and (re.search(r'[a-zA-Z]',word)):
          #   emission_prob[word][tag] = (word_tag_count[word][tag] - tag_count[tag]*0.79) / (tag_count[tag] + len(word_tag_count))
          # elif tag == "NOUN" and (re.search(r'ly$',word)):
          #   emission_prob[word][tag] = (word_tag_count[word][tag] + tag_count[tag]*0.2) / (tag_count[tag] + len(word_tag_count))
          # elif tag == "NOUN" and re.search(r'ness$',word):
          #   emission_prob[word][tag] = (word_tag_count[word][tag] + tag_count[tag]*0.8) / (tag_count[tag] + len(word_tag_count))
          # elif tag == "ADV" and (re.search(r'ly$',word)):
          #   emission_prob[word][tag] = (word_tag_count[word][tag] + tag_count[tag]*0.79) / (tag_count[tag] + len(word_tag_count))
          # # elif tag == "PRT" and ((re.search(r"'d$",word)) or re.search(r"'s$",word) or re.search(r"'ll$",word) or re.search(r"^'",word) or re.search(r"'ve$",word)):
          # #   emission_prob[word][tag] = (word_tag_count[word][tag] + tag_count[tag]*0.79) / (tag_count[tag] + len(word_tag_count))
          # # elif tag == "PRT" and (re.search(r"[']",word)):
          # #   emission_prob[word][tag] = (word_tag_count[word][tag] + tag_count[tag]*0.2) / (tag_count[tag] + len(word_tag_count))
          # else:
          #   emission_prob[word][tag] = (word_tag_count[word][tag] + 1) / (tag_count[tag] + len(word_tag_count))

    emission_prob_list.append(emission_prob)


In [9]:
def calc_transition_prob(data):
    tag_tag_count = defaultdict(Counter)
    transition_prob = defaultdict(dict)
    start_tag_count = Counter()
    tag_count = Counter()

    for sentence in data:
        prev_tag = start_tag
        for word, tag in sentence:
            tag_tag_count[prev_tag][tag] += 1
            prev_tag = tag
            tag_count[tag] += 1

    for prev_tag in tag_tag_count:
        total_count = sum(tag_tag_count[prev_tag].values())
        for curr_tag in tag_tag_count[prev_tag]:
            transition_prob[prev_tag][curr_tag] = (tag_tag_count[prev_tag][curr_tag] + 1) / (total_count + len(tag_tag_count))

    transition_prob_list.append(transition_prob)

In [10]:
import re

def is_number(s):
    if re.search(r'[$%]',s):
      return False

    # Check if the string contains at least one digit
    if re.search(r'[0-9]', s):
        if not re.search(r'[a-zA-Z]', s):
          return True

    return False

In [11]:
def is_written_number(s):
    # Define a set of common written numbers
    written_numbers = {
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine", "ten",
        "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen",
        "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety",
        "hundred", "thousand", "million", "billion", "trillion", 'by', "quintillion","sextillion","and","to","mid",
    }

    # Handle compound numbers like "twenty-one"
    s = s.lower().replace('-', ' ')
    parts = s.split()

    # Check if all parts are in the written numbers set
    return all(part in written_numbers or is_number(part) for part in parts)

In [12]:
def viterbi_algo(sentence, fold_index):
    sentence = sentence.copy()
    sentence.append('.')
    # if re.search(r'[a-zA-Z]',sentence[-1]):
    #   sentence.append('.')
    #   print("Sentence didn't ended with a .")
    viterbi_table = [{} for _ in range(len(sentence))]
    backpointer = [{} for _ in range(len(sentence))]
    emission_prob = emission_prob_list[fold_index]
    transition_prob = transition_prob_list[fold_index]

    for tag in transition_prob[start_tag]:
        word = sentence[0].lower()
        viterbi_table[0][tag] = transition_prob[start_tag].get(tag, 1e-6) * emission_prob.get(word, {}).get(tag, 1e-6)
        backpointer[0][tag] = start_tag

    for t in range(1, len(sentence)):
        word = sentence[t].lower()
        for curr_tag in transition_prob:
            max_prob, best_prev_tag = max(
                (viterbi_table[t-1][prev_tag] * transition_prob[prev_tag].get(curr_tag, 1e-6) * emission_prob.get(word, {}).get(curr_tag, 1e-6), prev_tag)
                for prev_tag in viterbi_table[t-1]
            )
            viterbi_table[t][curr_tag] = max_prob
            backpointer[t][curr_tag] = best_prev_tag

    best_path = []
    best_last_tag = max(viterbi_table[-1], key=viterbi_table[-1].get)
    best_path.append(best_last_tag)
    for t in range(len(sentence) - 1, 0, -1):
        best_last_tag = backpointer[t][best_last_tag]
        best_path.insert(0, best_last_tag)
    best_path.pop()
    return best_path

In [13]:
per_pos_accuracy = defaultdict(dict)

In [14]:
for each_tag1 in sorted_tags:
  for each_tag2 in sorted_tags:
    per_pos_accuracy[each_tag1][each_tag2] = 0

In [15]:
print(per_pos_accuracy)

defaultdict(<class 'dict'>, {'.': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'ADJ': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'ADP': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'ADV': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'CONJ': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'DET': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'NOUN': {'.': 0, 'ADJ': 0, 'ADP': 0, 'ADV': 0, 'CONJ': 0, 'DET': 0, 'NOUN': 0, 'NUM': 0, 'PRON': 0, 'PRT': 0, 'VERB': 0, 'X': 0, '^': 0}, 'NUM': {'.':

In [16]:
def calc_acc(actual, predicted):
    correct = sum(1 for a, p in zip(actual, predicted) if a == p)
    total = len(actual)
    for a,p in zip(actual, predicted):
      per_pos_accuracy[a][p] = per_pos_accuracy[a][p]+1
    return (correct / total) * 100

In [17]:
tag_predicted = []
actual_tags = []

for fold in range(K):
    train_data = [data_fold_wise[i] for i in range(K) if i != fold]
    train_data = [sentence for fold_data in train_data for sentence in fold_data]

    calc_emission_prob(train_data)
    calc_transition_prob(train_data)

    for sentence in data_fold_wise[fold]:
        input_words = [word for word, _ in sentence]
        true_tags = [tag for _, tag in sentence]
        best_tags = viterbi_algo(input_words, fold)

        actual_tags.extend(true_tags)
        tag_predicted.extend(best_tags)

    accuracy = calc_acc(actual_tags, tag_predicted)
    print(f"Fold {fold + 1} :{accuracy}")

overall_accuracy = calc_acc(actual_tags, tag_predicted)
print(f"Overall: {overall_accuracy}")


Fold 1 :93.86117335953759
Fold 2 :94.0681750371953
Fold 3 :94.2546987774055
Fold 4 :94.32776519277513
Fold 5 :94.33253264409177
Overall: 94.33253264409177


In [18]:
'''
s=[0,1,2]
print(s[-1])
if re.search(r'[a-zA-Z]','.kjjk'):
  print("True")
'''

'\ns=[0,1,2]\nprint(s[-1])\nif re.search(r\'[a-zA-Z]\',\'.kjjk\'):\n  print("True")\n'

In [19]:
print('\t',end="")
for tags in sorted_tags:
  print(tags,end='\t')
print()
for k1,p1 in per_pos_accuracy.items():
  print(k1, end='\t')
  for k2,p2 in p1.items():
    print(p2, end='\t')
  print()


	.	ADJ	ADP	ADV	CONJ	DET	NOUN	NUM	PRON	PRT	VERB	X	^	
.	593596	0	4	0	0	4	8	0	0	0	0	0	493	
ADJ	1317	321223	1260	9441	0	7192	15106	18	236	263	3160	0	228	
ADP	98	276	593714	7479	610	1791	318	5	1096	5615	574	0	483	
ADV	532	9942	8126	201017	305	1324	2403	0	363	2406	911	0	144	
CONJ	6	0	78	664	156643	432	31	0	0	0	2	0	201	
DET	5	4	2437	184	85	566366	34	4	3500	6	8	0	390	
NOUN	18455	20153	8443	700	3	17980	1087309	486	9943	484	17675	173	933	
NUM	817	441	367	0	0	1900	6033	56117	308	3	156	0	55	
PRON	12	4	7050	6	0	1448	182	0	175266	11	34	0	67	
PRT	45	265	13758	1220	0	101	1065	0	75	100586	190	0	80	
VERB	1818	5068	4467	371	0	5127	31254	0	190	22	694953	0	537	
X	453	383	445	20	16	592	2658	15	104	13	254	814	26	
^	0	0	0	0	0	0	0	0	0	0	0	0	0	


In [20]:
len(emission_prob_list)

5

In [22]:
sorted_tags

['.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X',
 '^']

In [23]:
per_pos_precision = defaultdict(dict)
for k1,p1 in per_pos_accuracy.items():
  if k1 != '^':
    i = 0
    for k2,p2 in p1.items():
      i += p2
    for k2,p2 in p1.items():
      per_pos_precision[k1][k2] = p2/i
print('\t',end="")
for tags in sorted_tags:
  print(tags,end='\t')
print()
for k1,p1 in per_pos_precision.items():
  print(k1, end='\t')
  for k2,p2 in p1.items():
    print(round(p2,2), end='\t')
  print()

	.	ADJ	ADP	ADV	CONJ	DET	NOUN	NUM	PRON	PRT	VERB	X	^	
.	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	
ADJ	0.0	0.89	0.0	0.03	0.0	0.02	0.04	0.0	0.0	0.0	0.01	0.0	0.0	
ADP	0.0	0.0	0.97	0.01	0.0	0.0	0.0	0.0	0.0	0.01	0.0	0.0	0.0	
ADV	0.0	0.04	0.04	0.88	0.0	0.01	0.01	0.0	0.0	0.01	0.0	0.0	0.0	
CONJ	0.0	0.0	0.0	0.0	0.99	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	
DET	0.0	0.0	0.0	0.0	0.0	0.99	0.0	0.0	0.01	0.0	0.0	0.0	0.0	
NOUN	0.02	0.02	0.01	0.0	0.0	0.02	0.92	0.0	0.01	0.0	0.01	0.0	0.0	
NUM	0.01	0.01	0.01	0.0	0.0	0.03	0.09	0.85	0.0	0.0	0.0	0.0	0.0	
PRON	0.0	0.0	0.04	0.0	0.0	0.01	0.0	0.0	0.95	0.0	0.0	0.0	0.0	
PRT	0.0	0.0	0.12	0.01	0.0	0.0	0.01	0.0	0.0	0.86	0.0	0.0	0.0	
VERB	0.0	0.01	0.01	0.0	0.0	0.01	0.04	0.0	0.0	0.0	0.93	0.0	0.0	
X	0.08	0.07	0.08	0.0	0.0	0.1	0.46	0.0	0.02	0.0	0.04	0.14	0.0	


In [24]:
viterbi_algo(['I','live','in','Mumbai'],1)

['PRON', 'VERB', 'ADP', 'NOUN']

In [25]:
len(data)

57340

In [26]:
emission_prob_list[0]['i']

{'PRON': 0.054311108981576865,
 'NOUN': 3.185930929017459e-05,
 'X': 4.60182692528934e-05}

In [27]:
import json
# Convert to JSON
json_data = json.dumps(emission_prob_list, indent=4)

# Save to a file
with open('emission_prob_list.json', 'w') as f:
    f.write(json_data)

In [28]:
import json

# Load JSON from file
with open('emission_prob_list.json', 'r') as f:
    json_data = f.read()

# Convert JSON to Python object
emission_prob_list = json.loads(json_data)

In [29]:
import json
# Convert to JSON
json_data = json.dumps(transition_prob_list, indent=4)

# Save to a file
with open('transition_prob_list.json', 'w') as f:
    f.write(json_data)

In [30]:
transition_prob_list[0]['^']["NOUN"]

0.1298358797759421

In [31]:
import json

# Load JSON from file
with open('transition_prob_list.json', 'r') as f:
    json_data = f.read()

# Convert JSON to Python object
transition_prob_list = json.loads(json_data)

In [32]:
transition_prob_list[0]['^']["NOUN"]

0.1298358797759421

Real Testing Code

In [43]:
def print_results(sentence, num_folds):
    # Convert sentence to list of words
    input_words = re.findall(r"\w+|[^\w\s]", sentence)
    # Dictionary to hold tags for each word position across all folds
    tags_by_position = defaultdict(list)

    # Collect POS tags from each fold
    for fold in range(num_folds):
        pos_tags = viterbi_algo(input_words, fold)

        for position, tag in enumerate(pos_tags):
            tags_by_position[position].append(tag)

    # Determine the most probable tag for each position
    most_probable_tags = []
    for position in range(len(input_words)):
        # Count the frequency of each tag at this position
        tag_counts = Counter(tags_by_position[position])
        # Get the most common tag
        most_common_tag, _ = tag_counts.most_common(1)[0]
        most_probable_tags.append(most_common_tag)

    # Print results
    for word, tag in zip(input_words, most_probable_tags):
        print(f"{word}: {tag}")

In [44]:
import json
import numpy as np
from nltk.tag import map_tag
from collections import defaultdict, Counter

start_tag = '^'
# Load JSON from file
with open('emission_prob_list.json', 'r') as f:
    json_data = f.read()

# Convert JSON to Python object
emission_prob_list = json.loads(json_data)
num_folds = len(emission_prob_list)
# Load JSON from file
with open('transition_prob_list.json', 'r') as f:
    json_data = f.read()

# Convert JSON to Python object
transition_prob_list = json.loads(json_data)
sentence = "This is the course."

print_results(sentence,5)

This: DET
is: VERB
the: DET
course: NOUN
.: .


In [45]:
print(num_folds)

5


In [47]:
#To do  !!!! important
sentence ="But, soft! what light through yonder window breaks? It is the east, and Juliet is the sun."
print_results(sentence,len(emission_prob_list))

But: CONJ
,: .
soft: ADJ
!: .
what: DET
light: NOUN
through: ADP
yonder: DET
window: NOUN
breaks: VERB
?: .
It: PRON
is: VERB
the: DET
east: NOUN
,: .
and: CONJ
Juliet: NOUN
is: VERB
the: DET
sun: NOUN
.: .
